# Add Flexibility to Calculation of Allocations

Thu 13 Sep, 2018

In [157]:
import numpy as np
import pandas as pd

## Create DataFrames with test data

In [158]:
#Note that in order fo the patronage calculations to work, this DataFrame needs to contain all members.
prev_equity_df = pd.DataFrame([
    [1, 'Alice', 0, 0, 0],
    [2, 'Bob', 125, 0, 0],
    [3, 'Candice', 125, 3000, 200],
    [4, 'Darwin', 0, 0, 0],
    [5, 'Eve', 125, 1000, 100],
    [6, 'Federico', 125, 0, 700],
    [7, 'Gabi', 125, 0, 0],
    [8, 'HAL', 125, 0, 0],
    [9, 'Irene', 125, 10000, 0],
    [10, 'Jules', 0, 0, 0]
], columns = ['member_id', 'name', 'membership', 'preferred', 'other'])
prev_equity_df['equity'] = prev_equity_df['membership'] + prev_equity_df['preferred'] + prev_equity_df['other']
prev_equity_df

,member_id,name,membership,preferred,other,equity
0,1,Alice,0,0,0,0
1,2,Bob,125,0,0,125
2,3,Candice,125,3000,200,3325
3,4,Darwin,0,0,0,0
4,5,Eve,125,1000,100,1225
5,6,Federico,125,0,700,825
6,7,Gabi,125,0,0,125
7,8,HAL,125,0,0,125
8,9,Irene,125,10000,0,10125
9,10,Jules,0,0,0,0


In [159]:
contributions_df = pd.DataFrame([
    [1, "1/3/2017", 125, 'membership'],
    [2, "1/31/2017", 400, 'other'],
    [1, "2/20/2017", 5000, 'preferred'],
    [3, "3/15/2017", -2000, 'preferred'],
    [1, "4/13/2017", 500, 'other'],
    [2, "4/13/2017", 7500, 'preferred'],
    [4, "6/24/2017", 125, 'membership'],
    [5, "7/17/2017", 3000, "preferred"],
    [6, "8/22/2017", -700, 'other'],
    [4, "10/18/2017", 20000, 'preferred'],
    [6, "11/11/2017", 1500, 'preferred'],
    [10, "5/4/2017", 125, 'membership'],
    [2, "9/20/2017", 200, 'other']
], columns = ['member_id','date','amount','type'])
contributions_df

,member_id,date,amount,type
0,1,1/3/2017,125,membership
1,2,1/31/2017,400,other
2,1,2/20/2017,5000,preferred
3,3,3/15/2017,-2000,preferred
4,1,4/13/2017,500,other
5,2,4/13/2017,7500,preferred
6,4,6/24/2017,125,membership
7,5,7/17/2017,3000,preferred
8,6,8/22/2017,-700,other
9,4,10/18/2017,20000,preferred


In [160]:
#The pivot_table function will be useful for converting a list of
#capital contributions to a 'previous equity' dataframe up to a given year.
contributions_df.pivot_table(index=['member_id'], columns=['type'], values='amount', aggfunc=np.sum, fill_value=0)

type,membership,other,preferred
member_id,,,
1,125,500,5000
2,0,600,7500
3,0,0,-2000
4,125,0,20000
5,0,0,3000
6,0,-700,1500
10,125,0,0


## Collect core functions from dividends notebook

Change percents to proportions.

In [197]:
def fraction_year_remaining(date):
    """Computes the fraction of the year remaining from a given date.
    """
    date = pd.to_datetime(date)
    offset = pd.tseries.offsets.YearBegin()
    year = date - offset
    next_year = date + offset
    #print(offset, year, next_year)
    return (next_year - date).days / (next_year - year).days

def compute_transaction_patronage(contributions_df):
    """Compute patronage for each transaction based on amount and fraction of year remaining.
    """
    contributions_df = contributions_df.copy()
    patronage = contributions_df['amount']*contributions_df['date'].apply(fraction_year_remaining)
    contributions_df['patronage'] = patronage
    return contributions_df

def compute_new_patronage(contributions_df):
    """Compute each member's patronage from new contributions for the current year.
    """
    contributions_df = compute_transaction_patronage(contributions_df)
    return contributions_df[['member_id','patronage']].groupby(by='member_id').sum()

def compute_patronage(prev_equity_df, contributions_df):
    """Compute total patronage for each member from new contributions for the current year
        and existing equity from previous years.
    """
    patronage_df = prev_equity_df.set_index('member_id')[['name', 'equity']]
    patronage_df.rename(columns={'equity': 'old_patronage'}, inplace=True)
    
    patronage_df['new_patronage'] = compute_new_patronage(contributions_df)['patronage']
    # If there were members with no contributions this year, set their new patronage to 0 (would be NaN).
    patronage_df.fillna(0, inplace=True)
    
    patronage_df['patronage'] = patronage_df['old_patronage'] + patronage_df['new_patronage']
    patronage_df['proportionate_patronage'] = patronage_df['patronage'] / patronage_df['patronage'].sum()
    
    return patronage_df

def compute_dividends(patronage_df, profit, proportion_individual=0.5, rounded=True):
    """Compute each member's dividend based on patronage for the year.
    """ 
    dividend_df = patronage_df[['name', 'proportionate_patronage']].copy()
    
    #Compute individual patronage allocations
    dividend_df['dividend'] = dividend_df['proportionate_patronage'] * profit * proportion_individual
    if rounded:
        dividend_df['dividend'] = np.round(dividend_df['dividend'], 2)
    
    # To account for rounding amounts to the nearest cent, we add up the individual dividends
    # to get the actual amount allocated to individual net income. Then we subtract this amount
    # from the total profit to get the collective net income.
    indiv_profit = dividend_df['dividend'].sum()
    collective_profit = profit - indiv_profit
    
    # We reserve member_id=0 for the collective account (or we could simply use names as keys)
    dividend_df.loc[0] = pd.Series({
        'name': 'CollectiveAcct',
        'proportionate_patronage': collective_profit / indiv_profit,
        'dividend': collective_profit
    })
    return dividend_df

## Auxiliary functions

In [162]:
def compute_total_patronage(patronage_df):
    """Computes the total patronage for the year, i.e. the time-averaged amount of equity.
    """
    return patronage_df['patronage'].sum()

def estimate_indiv_collective_profit(profit, proportion_individual):
    """Estimate the individual and collective net income from the total net income.
    This may be different from the actual individual and collective amounts because
    of rounding individual dividends to the nearest cent.
    """
    indiv_profit = np.round(profit * proportion_individual, 2)
    collective_profit = profit - indiv_profit
    return indiv_profit, collective_profit

In [163]:
patronage_df = compute_patronage(prev_equity_df, contributions_df)
patronage_df

,name,old_patronage,new_patronage,patronage,proportionate_patronage
member_id,,,,,
1,Alice,0,4799.657534,4799.657534,0.157379
2,Bob,125,5827.671233,5952.671233,0.195186
3,Candice,3325,-1600.000000,1725.000000,0.056562
4,Darwin,0,4175.000000,4175.000000,0.136897
5,Eve,1225,1380.821918,2605.821918,0.085444
6,Federico,825,-43.561644,781.438356,0.025623
7,Gabi,125,0.000000,125.000000,0.004099
8,HAL,125,0.000000,125.000000,0.004099
9,Irene,10125,0.000000,10125.000000,0.331995


In [164]:
patronage_df.sum()

name                       AliceBobCandiceDarwinEveFedericoGabiHALIreneJules
old_patronage                                                          15875
new_patronage                                                        14622.5
patronage                                                            30497.5
proportionate_patronage                                                    1
dtype: object

In [165]:
profit = 2182.33
#Different profit values used to test rounding behavior:
#2182.33, 0.04, 0.23, 1.17, 1729.45, 100934.27, 10.75, 3.27
#Note: 0.01, 0.02, 0.03 all result in 0.00 individual profit,
#leading to a divide by zero error for the collective proportion.
dividend_df = compute_dividends(patronage_df, profit)
dividend_df

,name,proportionate_patronage,dividend
member_id,,,
1,Alice,0.157379,171.73
2,Bob,0.195186,212.98
3,Candice,0.056562,61.72
4,Darwin,0.136897,149.38
5,Eve,0.085444,93.23
6,Federico,0.025623,27.96
7,Gabi,0.004099,4.47
8,HAL,0.004099,4.47
9,Irene,0.331995,362.26


In [166]:
estimate_indiv_collective_profit(profit, 0.5)

(1091.1600000000001, 1091.1699999999998)

In [167]:
dividend_df.sum()

name                       AliceBobCandiceDarwinEveFedericoGabiHALIreneJu...
proportionate_patronage                                              1.99999
dividend                                                             2182.33
dtype: object

## Test calculations with numpy arrays

In [168]:
a = np.array([0.25 for x in range(3)])
a

array([ 0.25,  0.25,  0.25])

In [169]:
a[0] = 0.5
a

array([ 0.5 ,  0.25,  0.25])

In [170]:
a * 4

array([ 2.,  1.,  1.])

In [171]:
a.reshape(1,-1) #Reshapes a to a 1x3 row vector

array([[ 0.5 ,  0.25,  0.25]])

In [172]:
#.reshape(-1,1) reshapes dividend_df['dividend'].values to a 11x1 column vector.
#The broadcast product with a.reshape(1,-1) will be 11x3.
b = a.reshape(1,-1) * dividend_df['dividend'].values.reshape(-1,1)
b

array([[  85.865 ,   42.9325,   42.9325],
       [ 106.49  ,   53.245 ,   53.245 ],
       [  30.86  ,   15.43  ,   15.43  ],
       [  74.69  ,   37.345 ,   37.345 ],
       [  46.615 ,   23.3075,   23.3075],
       [  13.98  ,    6.99  ,    6.99  ],
       [   2.235 ,    1.1175,    1.1175],
       [   2.235 ,    1.1175,    1.1175],
       [ 181.13  ,   90.565 ,   90.565 ],
       [   1.485 ,    0.7425,    0.7425],
       [ 545.58  ,  272.79  ,  272.79  ]])

In [173]:
b.sum()

2182.3299999999999

In [174]:
b.sum(axis=1)

array([  171.73,   212.98,    61.72,   149.38,    93.23,    27.96,
           4.47,     4.47,   362.26,     2.97,  1091.16])

In [175]:
#The row sums should equal the dividends:
b.sum(axis=1) == dividend_df['dividend']

member_id
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
0     True
Name: dividend, dtype: bool

In [176]:
np.round(b, 2)

array([[  85.86,   42.93,   42.93],
       [ 106.49,   53.24,   53.24],
       [  30.86,   15.43,   15.43],
       [  74.69,   37.34,   37.34],
       [  46.62,   23.31,   23.31],
       [  13.98,    6.99,    6.99],
       [   2.24,    1.12,    1.12],
       [   2.24,    1.12,    1.12],
       [ 181.13,   90.56,   90.56],
       [   1.48,    0.74,    0.74],
       [ 545.58,  272.79,  272.79]])

In [177]:
np.round(b, 2).sum()

2182.3099999999999

In [178]:
np.round(b, 2).sum(axis=1)

array([  171.72,   212.97,    61.72,   149.37,    93.24,    27.96,
           4.48,     4.48,   362.25,     2.96,  1091.16])

In [179]:
#After rounding, the row sums no longer equal the dividends:
np.round(b, 2).sum(axis=1) == dividend_df['dividend']

member_id
1     False
2     False
3      True
4     False
5     False
6      True
7     False
8     False
9     False
10    False
0      True
Name: dividend, dtype: bool

In [180]:
np.array([1,2,3] if False else [5,6,7,8]) #Woo hoo, inline if statements!

array([5, 6, 7, 8])

In [181]:
b

array([[  85.865 ,   42.9325,   42.9325],
       [ 106.49  ,   53.245 ,   53.245 ],
       [  30.86  ,   15.43  ,   15.43  ],
       [  74.69  ,   37.345 ,   37.345 ],
       [  46.615 ,   23.3075,   23.3075],
       [  13.98  ,    6.99  ,    6.99  ],
       [   2.235 ,    1.1175,    1.1175],
       [   2.235 ,    1.1175,    1.1175],
       [ 181.13  ,   90.565 ,   90.565 ],
       [   1.485 ,    0.7425,    0.7425],
       [ 545.58  ,  272.79  ,  272.79  ]])

In [182]:
b.round(2)

array([[  85.86,   42.93,   42.93],
       [ 106.49,   53.24,   53.24],
       [  30.86,   15.43,   15.43],
       [  74.69,   37.34,   37.34],
       [  46.62,   23.31,   23.31],
       [  13.98,    6.99,    6.99],
       [   2.24,    1.12,    1.12],
       [   2.24,    1.12,    1.12],
       [ 181.13,   90.56,   90.56],
       [   1.48,    0.74,    0.74],
       [ 545.58,  272.79,  272.79]])

In [183]:
b

array([[  85.865 ,   42.9325,   42.9325],
       [ 106.49  ,   53.245 ,   53.245 ],
       [  30.86  ,   15.43  ,   15.43  ],
       [  74.69  ,   37.345 ,   37.345 ],
       [  46.615 ,   23.3075,   23.3075],
       [  13.98  ,    6.99  ,    6.99  ],
       [   2.235 ,    1.1175,    1.1175],
       [   2.235 ,    1.1175,    1.1175],
       [ 181.13  ,   90.565 ,   90.565 ],
       [   1.485 ,    0.7425,    0.7425],
       [ 545.58  ,  272.79  ,  272.79  ]])

In [184]:
dividend_df['dividend'] - b.round(2)[:,1:].sum(axis=1)

member_id
1      85.87
2     106.50
3      30.86
4      74.70
5      46.61
6      13.98
7       2.23
8       2.23
9     181.14
10      1.49
0     545.58
Name: dividend, dtype: float64

In [185]:
#Make first year payments irregular
c = b.round(2)
c[:,0] = np.round(dividend_df['dividend'] - c[:,1:].sum(axis=1),2)
c

array([[  85.87,   42.93,   42.93],
       [ 106.5 ,   53.24,   53.24],
       [  30.86,   15.43,   15.43],
       [  74.7 ,   37.34,   37.34],
       [  46.61,   23.31,   23.31],
       [  13.98,    6.99,    6.99],
       [   2.23,    1.12,    1.12],
       [   2.23,    1.12,    1.12],
       [ 181.14,   90.56,   90.56],
       [   1.49,    0.74,    0.74],
       [ 545.58,  272.79,  272.79]])

In [186]:
c.sum()

2182.3299999999999

In [187]:
#Make last year payments irregular
d = b.round(2)
d[:,-1] = dividend_df['dividend'] - d[:,:-1].sum(axis=1)
d

array([[  85.86,   42.93,   42.94],
       [ 106.49,   53.24,   53.25],
       [  30.86,   15.43,   15.43],
       [  74.69,   37.34,   37.35],
       [  46.62,   23.31,   23.3 ],
       [  13.98,    6.99,    6.99],
       [   2.24,    1.12,    1.11],
       [   2.24,    1.12,    1.11],
       [ 181.13,   90.56,   90.57],
       [   1.48,    0.74,    0.75],
       [ 545.58,  272.79,  272.79]])

In [188]:
len(d)

11

## New function for calculating yearly allocations

In [247]:
def compute_allocations(dividend_df,
                        year,
                        first_year_proportion=0.5,
                        n_years=3,
                        distribution = None,
#                         pos_distribution = None,
#                         neg_distribution = None,
                        irregular_payment='last'):
    """Computes allocations over next n_years years after the dividend year.
    """
    
    #Currently we assume below that either all dividends will be positive or all will
    #be negative, depending on the overall profit; if this is always the case, then we
    #should really just have one distribution array instead of two.
    #Conceivably there could be a situation where some dividends
    #could be positive and some could be negative, in which case we'd need both arrays,
    #so I'll leave them both for now, though I can't currently think of why we would
    #want to allow that.
    
    #If no explicit payment distribution was passed, we need to create one.
    if distribution is None:
        #Check whether the total profit is positive or negative, and create
        #the appropriate default distribution.
        if dividend_df['dividend'].sum() >= 0:
            #profit >= 0
            #Default for a positive dividend is to evenly evenly divide what's left over after
            #the first year over the remaining n-1 years.
            if n_years == 1:
                #Avoid divide by zero error.
                distribution = [1]
            else:
                #n_years > 1
                distribution = [(1-first_year_proportion) / (n_years-1) for _ in range(n_years)]
                distribution[0] = first_year_proportion
        else:
            #profit < 0
            #Default for a negative dividend is to evenly divide it over all n years.
            distribution = [1.0 / n_years for _ in range(n_years)]
            
    #Convert the distribution from a list to a numpy array to perform math with it.
    distribution = np.array(distribution)
    
#     if pos_distribution is None:
#         #Default for a positive dividend is to evenly evenly divide what's left over after
#         #the first year over the remaining n-1 years.
#         if n_years > 1:
#             pos_distribution = [(1-first_year_proportion) / (n_years-1) for _ in range(n_years)]
#             pos_distribution[0] = first_year_proportion
#         else:
#             #Avoid divide by zero error.
#             pos_distribution = [1]
            
#     if neg_distribution is None:
#         #Default for a negative dividend is to evenly divide it over all n years.
#         neg_distribution = [1.0 / n_years for _ in range(n_years)]
    
#     #Check whether the profit is positive or negative, and use the corresponding distribution;
#     #convert the distribution from a list to a numpy array to perform math with it.
#     profit = dividend_df['dividend'].sum()
#     distribution = np.array(pos_distribution if profit >= 0 else neg_distribution)
    
    #In case the distribution was explicitly passed, make sure n_years matches the actual length.
    n_years = len(distribution)
    
    #Create a new DataFrame for the allocations by dropping the collective account and
    #the proportionate patronage column from dividend_df.
    allocation_df = dividend_df.drop(index=0, columns='proportionate_patronage')
    
    #Use broadcasting to compute all dividends with one multiplication,
    #and round them to nearest cent.
    allocations = (distribution.reshape(1,-1) * allocation_df['dividend'].values.reshape(-1,1)).round(2)
    
    #Adjust the first or last payment to account for rounding, by replacing the
    #first or last column, respectively, with the result of subtracting
    #the sum of the remaining payments from the member's actual dividend.
    #Or if they passed a specific year, adjust the payment for that year.
    if irregular_payment == 'first':
        allocations[:,0] = np.round(allocation_df['dividend'] - allocations[:,1:].sum(axis=1), 2)
    elif irregular_payment == 'last':
        allocations[:,-1] = np.round(allocation_df['dividend'] - allocations[:,:-1].sum(axis=1), 2)
    elif 1 <= irregular_payment <= n_years:
        sum_of_remaining = (allocations[:,:irregular_payment-1].sum(axis=1)
                            + allocations[:,irregular_payment:].sum(axis=1))
        allocations[:,irregular_payment-1] = np.round(allocation_df['dividend'] - sum_of_remaining, 2)
    else:
        raise ValueError("irregular_payment must be 'first', 'last', or an integer between 1 and n_years (inclusive).")
        
    #Create new column labels for the years year+1, year+2,...,year+n
    new_columns = [str(y) for y in range(year+1, year+n_years+1)]
    
    #Concatenate the existing allocation dataframes horizontally with the computed allocations.
    allocation_df = pd.concat([allocation_df,
                              pd.DataFrame(allocations, index=allocation_df.index, columns=new_columns)],
                             axis=1)
    
    #Rename the 'dividend' column.
    allocation_df.rename(columns={'dividend': str(year)+'_dividend'}, inplace=True)
    
    return allocation_df

In [194]:
allocation_df = compute_allocations(dividend_df, 2017, first_year_proportion=0.7, n_years=3)
allocation_df

,name,2017_dividend,2018,2019,2020
member_id,,,,,
1,Alice,171.73,120.21,25.76,25.76
2,Bob,212.98,149.09,31.95,31.94
3,Candice,61.72,43.20,9.26,9.26
4,Darwin,149.38,104.57,22.41,22.40
5,Eve,93.23,65.26,13.98,13.99
6,Federico,27.96,19.57,4.19,4.20
7,Gabi,4.47,3.13,0.67,0.67
8,HAL,4.47,3.13,0.67,0.67
9,Irene,362.26,253.58,54.34,54.34


In [195]:
allocation_df.sum()

name             AliceBobCandiceDarwinEveFedericoGabiHALIreneJules
2017_dividend                                              1091.17
2018                                                        763.82
2019                                                        163.68
2020                                                        163.67
dtype: object

In [196]:
allocation_df.loc[:,'2018':].sum(axis=1)

member_id
1     171.73
2     212.98
3      61.72
4     149.38
5      93.23
6      27.96
7       4.47
8       4.47
9     362.26
10      2.97
dtype: float64

## Compare rounding dividends vs. waiting until computing allocations to round

In [199]:
unrounded_dividend_df = compute_dividends(patronage_df, profit, rounded = False)
unrounded_dividend_df

,name,proportionate_patronage,dividend
member_id,,,
1,Alice,0.157379,171.726345
2,Bob,0.195186,212.979877
3,Candice,0.056562,61.718559
4,Darwin,0.136897,149.376801
5,Eve,0.085444,93.233375
6,Federico,0.025623,27.958985
7,Gabi,0.004099,4.472359
8,HAL,0.004099,4.472359
9,Irene,0.331995,362.261104


In [200]:
dividend_df

,name,proportionate_patronage,dividend
member_id,,,
1,Alice,0.157379,171.73
2,Bob,0.195186,212.98
3,Candice,0.056562,61.72
4,Darwin,0.136897,149.38
5,Eve,0.085444,93.23
6,Federico,0.025623,27.96
7,Gabi,0.004099,4.47
8,HAL,0.004099,4.47
9,Irene,0.331995,362.26


In [201]:
late_allocation_df = compute_allocations(unrounded_dividend_df, 2017, first_year_proportion=0.5, n_years=3)
late_allocation_df

,name,2017_dividend,2018,2019,2020
member_id,,,,,
1,Alice,171.726345,85.86,42.93,42.94
2,Bob,212.979877,106.49,53.24,53.25
3,Candice,61.718559,30.86,15.43,15.43
4,Darwin,149.376801,74.69,37.34,37.35
5,Eve,93.233375,46.62,23.31,23.30
6,Federico,27.958985,13.98,6.99,6.99
7,Gabi,4.472359,2.24,1.12,1.11
8,HAL,4.472359,2.24,1.12,1.11
9,Irene,362.261104,181.13,90.57,90.56


In [202]:
early_allocation_df = compute_allocations(dividend_df, 2017, first_year_proportion=0.5, n_years=3)
early_allocation_df

,name,2017_dividend,2018,2019,2020
member_id,,,,,
1,Alice,171.73,85.86,42.93,42.94
2,Bob,212.98,106.49,53.24,53.25
3,Candice,61.72,30.86,15.43,15.43
4,Darwin,149.38,74.69,37.34,37.35
5,Eve,93.23,46.62,23.31,23.30
6,Federico,27.96,13.98,6.99,6.99
7,Gabi,4.47,2.24,1.12,1.11
8,HAL,4.47,2.24,1.12,1.11
9,Irene,362.26,181.13,90.56,90.57


The only difference I see between the late-rounded allocations and the early-rounded allocations is that **Irene's second and third payment amounts are switched.**

Let's add up the amounts of the late-rounded allocations to compute the late-rounded dividends, and see if they agree with the early-rounded dividends:

In [206]:
late_allocation_df.loc[:,'2018':].sum(axis=1)

member_id
1     171.73
2     212.98
3      61.72
4     149.38
5      93.23
6      27.96
7       4.47
8       4.47
9     362.26
10      2.97
dtype: float64

In [207]:
#They look the same as the rounded dividends... let's compare:
late_allocation_df.loc[:,'2018':].sum(axis=1) == early_allocation_df['2017_dividend']

member_id
1      True
2      True
3      True
4      True
5     False
6      True
7     False
8     False
9      True
10    False
dtype: bool

**Some compaisons are false.** Let's print the full floating point numbers to see how close they are:

In [210]:
late_allocation_df.loc[5,'2018':].sum()

93.22999999999999

In [211]:
early_allocation_df.loc[5,'2017_dividend']

93.230000000000004

In [213]:
late_allocation_df.loc[7,'2018':].sum()

4.4700000000000006

In [214]:
early_allocation_df.loc[7,'2017_dividend']

4.4699999999999998

Ok, so it looks like the **differences are due to the machine accuracy**, and not due to rounding at different stages, though I don't know if that's always guaranteed...

Let's see what happens if we round both results to two decimals:

In [215]:
np.round(late_allocation_df.loc[:,'2018':].sum(axis=1), 2) == np.round(early_allocation_df['2017_dividend'], 2)

member_id
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
dtype: bool

In [230]:
np.round(late_allocation_df.loc[5,'2018':].sum(), 2)

93.230000000000004

In [231]:
np.round(early_allocation_df.loc[5,'2017_dividend'], 2)

93.230000000000004

Great, so the computer can actually tell that they're the same within two decimal places.

## Test some list indexing stuff to allow more options for the irregular payment

In [218]:
[0,1,2,3,4][:3]

[0, 1, 2]

In [225]:
[0,1,2,3,4][:2] + [0,1,2,3,4][3:]

[0, 1, 3, 4]

In [229]:
[0,1,2,3,4][:5] + [0,1,2,3,4][5:]

[0, 1, 2, 3, 4]

In [235]:
4<6<6<7

False

In [236]:
[1,2,3,4,5][:2] + [1,2,3,4,5][3:]

[1, 2, 4, 5]

In [239]:
np.array([1,2,3,4,5])[:2] + np.array([1,2,3,4,5])[3:]

array([5, 7])

In [248]:
#Specify that the irregular payment should happen in the second year (rather than the first or last year):
compute_allocations(dividend_df, 2017, first_year_proportion=0.5, n_years=3, irregular_payment=2)

,name,2017_dividend,2018,2019,2020
member_id,,,,,
1,Alice,171.73,85.86,42.94,42.93
2,Bob,212.98,106.49,53.25,53.24
3,Candice,61.72,30.86,15.43,15.43
4,Darwin,149.38,74.69,37.35,37.34
5,Eve,93.23,46.62,23.30,23.31
6,Federico,27.96,13.98,6.99,6.99
7,Gabi,4.47,2.24,1.11,1.12
8,HAL,4.47,2.24,1.11,1.12
9,Irene,362.26,181.13,90.57,90.56
